# Homework 1 (2022) : Data Wrangling and Visualization

Due date : **2022-01-30 @23h55** (this is a **hard deadline**)

## Fill this cell with your names

- Musafiri, David, Mathématique-Informatique
- Haoud, Oussama, Informatique

## Carefully follow instructions given on the [course homepage](https://stephanegaiffas.github.io/big_data_course/homeworks/)

Write in English or French

- The deliverable is a `xxx.ipynb` file (jupyter notebook) or a `xxx.py` file (if you are using `jupytext`) built by completing the template. **We won't execute the code in your notebook:** all your results, displays and plots must be visible without having to rerun everything.

Once again, follow the steps described here : [course homepage](https://stephanegaiffas.github.io/big_data_course/homeworks/). **If you don't: no evaluation!**

## Grading <i class="fa graduation-cap"></i>

Here is the way we'll assess your work

| Criterion | Points | Details |
|:----------|:-------:|:----|
|Spelling and syntax | 3 | English/French  |
|Plots correction | 3 |  Clarity / answers the question  |
|Plot style and cleanliness | 3 | Titles, legends, labels, breaks ... |
|Table wrangling | 4 | ETL, SQL like manipulations |
|Computing Statistics | 5 | SQL `goup by`  and aggregation  |
|DRY compliance | 2 | DRY principle at [Wikipedia](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)|

If we see a single (or more) `for` loop in your code: **-5 points**.  Everything can be done using high-level `pandas` methods

# Preliminaries

## Notebook: Modus operandi

- This is a [Jupyter Notebook]().
- When you execute code within the notebook, the results appear beneath the code.
- [Jupytext]()

## Packages

- Base `Python` can do a lot. But the full power of `Python` comes from a fast growing collection of `packages`/`modules`.

- Packages are first installed (that is using `pip install` or `conda install`), and if
needed, imported during a session.

- The `docker` image you are supposed to use already offers a lot of packages. You should not need to install new packages.

- Once a package has been installed on your drive, if you want all objects exported by the package to be available in your session, you should import the package, using `from pkg import *`.

- If you just want to pick some sobjects from the package,
you can use qualified names like `pkg.object_name` to access the object (function, dataset, class...)


In [22]:
# importing basic tools
import numpy as np
import pandas as pd

from pandas.api.types import CategoricalDtype

import os            # file operations
import requests      # networking

from datetime import date  # if needed

In [23]:
# importing plotting packages
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

In [24]:
# make pandas plotly-friendly
np.set_printoptions(precision=2, suppress=True)
%matplotlib inline
pd.options.plotting.backend = "plotly"

# Getting the data

The data are built and made available by [INSEE](https://www.insee.fr/fr/accueil)  (French Governement Statistics Institute)

Prénoms:
- [https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip](https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip)

This dataset has been growing for a while. It has been considered by
social scientists for decades.  Given names are meant to give insights into a variety
of phenomena, including religious observance.

- A glimpse at the body of work can be found in [_L'archipel français_ by Jérome Fourquet, Le Seuil, 2019 ](https://www.seuil.com/ouvrage/l-archipel-francais-jerome-fourquet/9782021406023)

- Read the [File documentation](https://www.insee.fr/fr/statistiques/2540004?sommaire=4767262#documentation)

## Download the data

**QUESTION:** Download the data into a file which relative path is `'./nat2020_csv.zip'`

__Hints:__

- Have a look at [`requests`](https://requests.readthedocs.io/en/master/).
- Use magic commands to navigate across the file hierarchy and create subdirectories
when needed

In [25]:
params = dict(
    url = 'https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip',
    dirpath = './',
    timecourse = '',
    datafile = 'nat2020.hdf',
    fpath = 'nat2020_csv.zip'
)

In [26]:
# modify location  make sure you are in the right directory
# %cd
# %pwd  #
# %ls
# %mkdir # if needed

In [27]:
url = params['url']      # 'https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip'
fpath = params['fpath']  # './nat2020_csv.zip'

In [28]:
with requests.get(params['url']) as r:
    with open(params['fpath'], "wb") as f:
        f.write(r.content)

## Load the data in memory

**QUESTION:** Load the data in a `pandas` `DataFrame` called `data`

__Hints:__

- You should obtain a `Pandas dataframe` with 4 columns.
- Mind the conventions used to build theis `csv` file.
- Package `pandas` provides the convenient tools.
- The dataset, though not too large, is already demanding.
- Don't hesitate to test your methods on a sample of rows method `sample()` from class `DataFrame` can be helpful.

In [29]:
data = pd.read_csv(params['fpath'],compression='zip',sep=';')
print(data.head())

   sexe        preusuel annais  nombre
0     1  _PRENOMS_RARES   1900    1250
1     1  _PRENOMS_RARES   1901    1342
2     1  _PRENOMS_RARES   1902    1330
3     1  _PRENOMS_RARES   1903    1286
4     1  _PRENOMS_RARES   1904    1430


## Explore the data

**QUESTION:** Look at the data, Use the attributes `columns`, `dtypes` and the methods `head`, `describe`, to get a feeling of the data.

- This dataset is supposed to report all given names used
for either sex during a year in France since 1900

- The file is made of `652 056` lines and  4 columns.

```
|-- preusuel : object
|-- nombre: int64
|-- sexe: int64
|-- annais: object
```

Each row indicates for a given `preusuel` (prénom usuel, given name), `sexe` (sex), and `annais` (année naissance, birthyear) the `nombre` (number) of babies of the given sex who were given that name during the given year.

|sexe    |preusuel     | annais|   nombre|
|:------|:--------|----:|---:|
|2     |SYLVETTE | 1953| 577|
|1   |BOUBOU   | 1979|   4|
|1   |NILS     | 1959|   3|
|2   |NICOLE   | 2003|  36|
|1   |JOSÉLITO | 2013|   4|


**QUESTION:** Compare memory usage and disk space used by data

**Hints:**

- The method `info`  prints a concise summary of a `DataFrame`.
- With optional parameter `memory_usage`, you can get an estimate
of the amount of memory used by the `DataFrame`.
- Beware that the resulting estimate depends on the argument fed.

In [30]:
data.info(memory_usage=True)

%ls -l

#The memory used by the DataFrame is 20.4+ MB
#The disk space used by data (i.e the size of the .zip file) is 2,4 Mo

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667364 entries, 0 to 667363
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   sexe      667364 non-null  int64 
 1   preusuel  667362 non-null  object
 2   annais    667364 non-null  object
 3   nombre    667364 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 20.4+ MB
total 9972
-rw-r--r-- 1 root root  463908 Jan 30 23:11 'homework01 18.ipynb'
-rw-r--r-- 1 root root  470646 Jan 30 23:13 'homework01 19.ipynb'
-rw-r--r-- 1 root root  463681 Jan 30 23:15 'homework01 20.ipynb'
-rw-r--r-- 1 root root  501271 Jan 30 23:17 'homework01 21.ipynb'
-rw-r--r-- 1 root root  462225 Jan 30 23:19 'homework01 22.ipynb'
-rw-r--r-- 1 root root  464027 Jan 30 23:20 'homework01 23.ipynb'
-rw-r--r-- 1 root root  462178 Jan 30 23:21 'homework01 24.ipynb'
-rw-r--r-- 1 root root  459511 Jan 30 23:23 'homework01 25.ipynb'
-rw-r--r-- 1 root root  464030 Jan 30 23:25 'homework01 26.ipynb'


**QUESTION:** Display the output of `.describe()` with style.

In [31]:
# your code here

data.describe()

,sexe,nombre
count,667364.000000,667364.000000
mean,1.543828,129.772665
std,0.498076,884.912504
min,1.000000,1.000000
25%,1.000000,4.000000
50%,2.000000,8.000000
75%,2.000000,25.000000
max,2.000000,53584.000000


**QUESTION:** For each column compute the number of distinct values

In [32]:
data.nunique()

sexe            2
preusuel    35010
annais        122
nombre       7304
dtype: int64

# Transformations

## Improving the data types

**QUESTION:** Make `sexe` a category with two levels `Female` and `Male`. Call the new column `genre`. Do you see any reason why this factor should be ordered?

__Hint:__ Read [Pandas and categorical variables](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html?highlight=category)

In [33]:
data['genre'] = data['sexe'].astype("category")

In [34]:
data['genre'] = data['genre'].cat.rename_categories(['Male', 'Female'])

**QUESTION:** Compare memory usage of columns `sexe` and `genre`

In [35]:
# your code here

print(data['genre'].memory_usage())
print(data['sexe'].memory_usage())

data['genre']

667616
5339040


0           Male
1           Male
2           Male
3           Male
4           Male
           ...  
667359    Female
667360    Female
667361    Female
667362    Female
667363    Female
Name: genre, Length: 667364, dtype: category
Categories (2, object): ['Male', 'Female']

**QUESTION:** Would it be more memory-efficient to recode `sexe` using modalities `F` and `M` instead of `Male` and `Female` ?

_Insert your answer here_

> No because it's not the String 'Male'/'Female' that is stored but a value coresponding to the position of the category

## Dealing with missing values

**QUESTION:** Variable `annais` class is `object`. Make `annais` of type `float`. Note that missing years are encoded as "XXXX", find a way to deal with that.

__Hint:__  As of releasing this Homework (2021-01-22), `Pandas` is not very good at managing missing values,
see [roadmap](https://pandas.pydata.org/docs/development/roadmap.html). Don't try to convert `annais` into an integer column.

In [36]:
data['annais'] = pd.to_numeric(data['annais'], downcast='float', errors='coerce')

data['annais'] 


0         1900.0
1         1901.0
2         1902.0
3         1903.0
4         1904.0
           ...  
667359    2017.0
667360    2018.0
667361    2019.0
667362    2020.0
667363       NaN
Name: annais, Length: 667364, dtype: float32

## Rename and remove columns

**QUESTION:** Remove useless columns (now that you've created new ones, and rename them). You should end up with a dataframe with columns called `"gender"`, `"year"`, `"count"`, `"firstname`" with the following dtypes:

```python
gender        category
firstname     object
count         int64
year          float64
```

In [37]:
# your code here
data = data.drop(columns=["sexe"]).rename(columns={"preusuel":"firstname", "annais":"year", "genre":"gender", "nombre":"count"})
data

,firstname,year,count,gender
0,_PRENOMS_RARES,1900.0,1250,Male
1,_PRENOMS_RARES,1901.0,1342,Male
2,_PRENOMS_RARES,1902.0,1330,Male
3,_PRENOMS_RARES,1903.0,1286,Male
4,_PRENOMS_RARES,1904.0,1430,Male
...,...,...,...,...
667359,ZYNEB,2017.0,6,Female
667360,ZYNEB,2018.0,5,Female
667361,ZYNEB,2019.0,7,Female
667362,ZYNEB,2020.0,8,Female


**QUESTION:** How many missing values (NA) have been introduced? How many births are concerned?

In [38]:
# your code here

data[data['year'].isna()].shape[0]


36675

**QUESTION:** Read the documentation and describe the origin of rows containing the missing values.

> "Les effectifs des prénoms remplissant la condition 2 mais pas la condition 3 sont regroupés (pour chaque sexe et chaque prénom) dans un enregistrement dont le champ année de naissance (ANNAIS) prend la valeur «XXXX»."

> The rows containing missing values correpsond to the names that were given at least 20 times from 1946 to 2020 but weren't given at least 3 times each year.

## Checkpointing: save your transformed dataframe

**QUESTION:** Save the transformed dataframe (retyped and renamed) to `./nat2020_csv.zip`. Try several compression methods.

In [ ]:
data.to_csv(path_or_buf='./nat2020_csv.zip', compression='zip')

**QUESTION:** Save the transformed dataframe (retyped and renamed) to `./nat2020.hdf` using `.hdf` format

In [18]:
#This line doesn't work because the optional dependency 'tables' is missing

#data.to_hdf('./nat2020.hdf', key='prenoms', format='table')

At that point your working directory should look like:

```
├── homework01.py      # if you use `jupytext`
├── homework01.ipynb
├── nat2020.hdf
├── nat2020.zip
```

**QUESTION:** Reload the data using `read_hdf(...)` so that the resulting dataframe  is properly typed with meaningful column names.

__Hint:__ use `try: ... except` to handle exceptions such as `FileNotFoundError`

In [19]:
read_hdf('./nat2020.hdf')
%ls

NameError: name 'read_hdf' is not defined

## Some data "analytics" and visualization

**QUESTION**: For each year, compute the total number of Female and Male births and the proportion of Female  births among total births

__Hints:__

- Groupby operations using several columns for the groups return a dataframe with a `MultiIndex` index see [Pandas advanced](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html)

- Have a look at `MultiIndex`, `reset_index`, `pivot`, `columns.droplevel`

In [20]:
# your code here

data2 = data.groupby(['year', 'gender']).sum()
data2

NameError: name 'data' is not defined

In [21]:
data3 =  data[data['gender']=='Female'].groupby(['year']).sum() / data.groupby(['year']).sum()
data3 = data3.rename(columns={"count":"f_proportion"})
data3

NameError: name 'data' is not defined

**QUESTION:** Plot the proportion of female births as a function of year. Don't forget: title, axes labels, ticks, scales, etc.

Because of what we did before, the `plot` method of a `DataFrame` with be rendered using `plotly`, so you can use this. But you can use also `seaborn` or any other available plotting library that you want.

__Hint:__ Mind the missing values in the `year` column

In [108]:
data3.plot(y='f_proportion')

**QUESTION:** Make any sensible comment about this plot.

_Insert your answer here_

> As we can expect, the proportion of female births is around 0.5

**QUESTION:** Explore the fluctuations of sex ratio around its mean value since 1945.
Plot deviations of sex ratio around its mean since 1945 as a function of time.

In [188]:
female_birth_per_year = data[data['gender']=='Male'].groupby(['year']).sum()

data4 =  data[data['gender']=='Female'].groupby(['year']).sum() / female_birth_per_year
data4 = data4.rename(columns={"count":"sex_ratio"})
data4 = data4[1946:]
# We removed the 1945 year for scaling issues

data4.plot(y='sex_ratio')

# Popular firstnames

## The top-20 most popular firstnames since 1900

**QUESTION:** For each firstname and sex (some names may be given to girls and boys), compute the total number of times this firstname has been given during `1900-2019`. Print the top 20 firstnames given and style your result dataframe using `background_gradient` for instance.

In [17]:
df = data[data['year'] <= 2019].groupby(['firstname', 'gender']).sum()
df = df.sort_values(['count'], ascending = False).drop(columns=['year']).head(10)
df

,,count
firstname,gender,
MARIE,Female,2231586
JEAN,Male,1913439
PIERRE,Male,891177
_PRENOMS_RARES,Female,826278
MICHEL,Male,820288
_PRENOMS_RARES,Male,773079
ANDRÉ,Male,711845
JEANNE,Female,557620
PHILIPPE,Male,538427


## Rare firstnames

**QUESTION:** For each sex, plot the proportion of births given `_PRENOMS_RARES` as a function of the year.

In [110]:
df2 = data[data['firstname'] == "_PRENOMS_RARES"].groupby(['year', 'gender']).sum() 

df2['count'] = df2['count'] / data.groupby(['year', 'gender']).sum()['count']
df2.xs('Female', level=1).plot(kind='bar',title='Proportion of births given _PRENOMS_RARES as a function of the year (Female)')

In [117]:
df2.xs('Male', level=1).plot(kind='bar', title='Proportion of births given _PRENOMS_RARES as a function of the year (Male)')

# A study of the "Marie" firstname

**QUESTION:** Plot the proportion of female births given name `'MARIE'` or `'MARIE-...'` (compounded names) as a function of the year.
Proceed in such a way that the reader can see the share of compounded names. We are expecting an _area plot_.

__Hints:__

- Have a look at the `.str` accessor (to apply a string method over a whole column containing string)
- Have a look at [r-graph-gallery: stacked area](https://www.r-graph-gallery.com/stacked-area-graph.html)  and
at [ggplot documentation](https://ggplot2.tidyverse.org/reference/geom_ribbon.html). Pay attention on the way you stack the area corresponding to names matching pattern 'MARIE-.*' over or under the are corresponding to babies named 'MARIE'
- See Graphique 3, page 48, de _L'archipel français_  de J. Fourquet. Le Seuil. Essais. Vol. 898.

- Add annotation, 1st World War, Front Populaire, 2nd World War, 1968

In [1]:
# your code here
marie_df = data[(data['gender'] == 'Female') &(data['firstname'].notna()) & (data['firstname'] == "MARIE")]
marie_composed_df = data[(data['gender'] == 'Female') &(data['firstname'].notna()) & data['firstname'].str.contains("^MARIE-", regex=True)]
marie_composed_df = marie_composed_df.groupby(['year']).sum()


marie_df = marie_df.join(marie_composed_df, 'year', rsuffix='2').set_index('year')

marie_df['count'] = marie_df['count'] / female_birth_per_year['count']
marie_df['count2'] = marie_df['count2'] / female_birth_per_year['count']
marie_df = marie_df.rename(columns={"count":"proportion of Marie", "count2":"proportion of Marie-..."})
marie_df.plot.area(y=['proportion of Marie', 'proportion of Marie-...'], title="Proportion of female births given name Marie or Marie-...")



NameError: name 'data' is not defined

# Top 10 firstnames of year 2000

**QUESTION:** For each sex, select the ten most popular names in year 2000, and plot the proportion
of newborns given that name over time. Take into account that some names might have
zero occurrence during certain years.

__Hint:__ Leave aside the rows with '_PRENOMS_RARES'.

In [73]:
yr_2000 = data[(data['year']==2000) & (data['firstname'] != '_PRENOMS_RARES')].join(data[data['firstname'] != '_PRENOMS_RARES'].groupby(['year', 'gender']).sum(), on=['year', 'gender'], rsuffix='2')
m_popular_names_yr_2000 = yr_2000[yr_2000['gender'] == "Male"].sort_values('count', ascending=False).head(10)
f_popular_names_yr_2000 = yr_2000[yr_2000['gender'] == "Female"].sort_values('count', ascending=False).head(10)

onlytop_f = data[(data['year'].notna()) &(data['gender'] == 'Female') & (data['firstname'].isin(f_popular_names_yr_2000['firstname']))]
onlytop_f = onlytop_f.join(data[data['firstname'] != '_PRENOMS_RARES'].groupby(['year']).sum(), on=['year'], rsuffix='2')
onlytop_f['proportion'] = onlytop_f['count'] / onlytop_f['count2']
onlytop_f.plot(x='year', y='proportion', color='firstname', title='Proportion of newborns given one of the Top 10 firstnames of year 2000 (Female)') 

#data.groupby(['year']).sum()['count']
#onlytop_m['proportion'] = onlytop['count'] 

In [75]:
onlytop_m = data[(data['year'].notna()) & (data['gender'] == 'Male') & (data['firstname'].isin(m_popular_names_yr_2000['firstname']))]
onlytop_m = onlytop_m.join(data[data['firstname'] != '_PRENOMS_RARES'].groupby(['year']).sum(), on=['year'], rsuffix='2')
onlytop_m['proportion'] = onlytop_m['count'] / onlytop_m['count2']

onlytop_m

onlytop_m.plot(x='year', y='proportion', color='firstname', title='Proportion of newborns given one of the Top 10 firstnames of year 2000 (Male)') 

# Getting help

- [pandas](https://pandas.pydata.org/pandas-docs/stable/reference/)

- [plotly](https://plotly.com/python/) for animated plots

- [stackoverflow](https://stackoverflow.com)

- [stackoverflow: pandas](https://stackoverflow.com/questions/tagged/pandas)

- [stackoverflow: plotly+python](https://stackoverflow.com/questions/tagged/plotly+python)

- The US `babynames` analogue of the INSEE file has been a playground for data scientists,
 see [https://github.com/hadley/babynames](https://github.com/hadley/babynames)

- Don't Repeat Yourself (DRY) principle  at [Wikipedia](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)